PLOTTING PAC SOURCES

Algorithm to plot sources:
1. Iterate through subjects, for each:
    - get PAC data
    - transpose
    - impute NaNs
    - aggregate
2. Stack PAC estimates along subjects axis
3. Average across ph, amp and sub


4. Load any morphed source file:
    - average time domain to get 1 point
    - replace data with PAC data
    - rename subject to "fsaverage_bem"
    - PLOT as regular source estimate
    - (optional) add ROI, if needed

In [ ]:
import copy
import numpy as np
import os
from utils import check_paths

import mne
from mne.stats import permutation_cluster_1samp_test

import scipy
from scipy.stats import zscore
from scipy.sparse import coo_matrix, save_npz

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib qt

In [ ]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
groups = ['Y']
tasks = ['_BL', '_MAIN'] # ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
# block_names = ['_baseline', '_adaptation']

# Load adjacency matrix
src_fname = 'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y\\fsaverage_bem\\bem\\fsaverage-ico4-src.fif'
src = mne.read_source_spaces(src_fname)
# src.plot(subjects_dir='D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y')
source_adjacency = mne.spatial_src_adjacency(src)
adj = source_adjacency.tocsr()  # Ensure adjacency is CSR format for fast indexing
print('adjacency shape:', source_adjacency.shape)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 5124
adjacency shape: (5124, 5124)


In [ ]:
# Load one stc file as a reference
stc_path = os.path.join('''D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y\\s1_pac_sub01\\preproc\\analysis\\source\\morphed_stcs\\_MAIN\\_plan\\_baseline\\s1_pac_sub01_MAIN_plan_baseline-stc-lcmv_epoch_000_morphed-lh.stc''')
stc_ref = mne.read_source_estimate(stc_path, subject='s1_pac_sub01') # any STC will serve as a template
stc_ref_mean = stc_ref.copy().mean()
stc_pac = stc_ref_mean.copy()
stc_pac.subject = "fsaverage_bem"
stc_pac

<SourceEstimate | 5124 vertices, subject : fsaverage_bem, tmin : 1.0000000000000009 (ms), tmax : 1.0000000000000009 (ms), tstep : 1002.0 (ms), data shape : (5124, 1), ~60 KiB>

In [ ]:
# Plot PAC MI values

views = {'lateral': ["lateral", "medial"],
        'dor-vent': ["dorsal", "ventral"]}

for group in groups:
    subs = os.listdir(os.path.join(eeg_data_dir, group))
    figs_save_path = os.path.join(eeg_data_dir, f'{group} group', 'sources', 'figs')
    check_paths(figs_save_path)

    

    for task in tasks:
        if task == '_BL':
            block_names = ['']
        else:
            block_names = ['_baseline', '_adaptation']

        for task_stage in task_stages:
            for block_name in block_names:
                print(f'Processing {group} group, {task}{task_stage}{block_name}...')

                pac_list = []
                for sub_name in subs:
                    analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
                    pac_save_path = os.path.join(analysis_dir, 'source', 'PAC')
                    pac_data = np.load(os.path.join(pac_save_path, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"))
                    pac_data_t = np.transpose(pac_data, (1, 0, 2))

                    ### NAN Imputation for PAC Matrices ###
                    pac_imputed = pac_data_t.copy()

                    # Step 1: Detect vertices with any NaN in their 20×20 PAC
                    nan_mask = np.isnan(pac_data_t).any(axis=(1, 2))  # shape (5124,)

                    if nan_mask.any() == True:
                        # print(f"Found {nan_mask.sum()} vertices with NaN values.")

                        # Step 2: Impute NaNs from neighbors
                        for vtx in np.where(nan_mask)[0]:
                            neighbors = adj[[vtx]].indices
                            valid_neighbors = [n for n in neighbors if not nan_mask[n]]
                            # Average PAC matrices from neighbors
                            pac_imputed[vtx] = np.nanmean(pac_data_t[valid_neighbors], axis=0)

                    # pac_zscore = zscore(pac_imputed, axis=0, nan_policy='omit')
                    pac_list.append(pac_imputed)

                # Stack them along a new first axis (subject axis)
                pac_all = np.stack(pac_list, axis=0)
                # print('PAC array shape:', pac_all.shape)
                pac_map = pac_all.mean(axis=(0, 2, 3))
                # print('Averaged PAC map shape:', pac_map.shape)
                stc_pac._data = pac_map[:, None]   # (n_vertices_total, 1)
                # print('STC data shape:', stc_pac.data.shape)

                clim = dict(
                    kind="value",
                    lims=[
                        np.nanmin(pac_map), 
                        (np.nanmax(pac_map) - np.nanmin(pac_map)) / 2 + np.nanmin(pac_map), 
                        np.nanmax(pac_map)
                        ],   # [min, mid, max]
                    )

                for key, value in views.items():
                    title = f"ALL_SUBS_{group}{task}{task_stage}{block_name}_PAC_MI_{key}"

                    brain = stc_pac.plot(subject="fsaverage_bem",
                                initial_time=0.0,
                                clim=clim,
                                cortex="low_contrast",
                                colormap='plasma', # 'plasma' for raw MI, 'cool' for zscored MI
                                hemi='both',
                                views=value, # "dorsal", "ventral"
                                background='white',
                                title = title
                                )

                    brain.save_image(os.path.join(figs_save_path, title + '.png'))


Processing Y group, _BL_plan...
Processing Y group, _BL_go...
Processing Y group, _MAIN_plan_baseline...
Processing Y group, _MAIN_plan_adaptation...
Processing Y group, _MAIN_go_baseline...
Processing Y group, _MAIN_go_adaptation...


Using control points [0.         0.         0.00012973]
Using control points [0.        0.        0.0003008]


PLOTTING STATS

In [ ]:
# Plot significant PAC

# Significant conditions:
# FTT (BL) planning
# De-CRAT (MAIN) planning baseline

group_save_path = os.path.join(eeg_data_dir, f'{group} group')
pac_stats_save_path = os.path.join(group_save_path, 'source_pac_stats')
sig_conditions = ['Y_BL_plan', 'Y_MAIN_plan_baseline']
p_thresh = 0.05

for sig_cond in sig_conditions:

    # Load stats data
    T_obs = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{sig_cond}_freqs_ave_T_obs.npy"))
    clusters = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{sig_cond}_freqs_ave_clusters.npy"), allow_pickle=True)
    cluster_p_values = np.load(os.path.join(pac_stats_save_path, f"PAC_MI_SOURCE_{sig_cond}_freqs_ave_cluster_p_values.npy"))

    # Combine significant cluster masks
    sig_mask = np.zeros_like(T_obs, dtype=bool)
    for cluster, p_val in zip(clusters, cluster_p_values):
        if p_val <= p_thresh:
            sig_mask |= cluster.astype(bool)
    sig_mask = sig_mask[:, None]
    title = f"Significant PAC - {sig_cond}"

    sig_cond = [sig_cond + '_' if sig_cond == 'Y_BL_plan' else sig_cond][0]
    group, task, task_stage, block_name = sig_cond.split('_')
    block_name = ['_' + block_name if block_name != '' else block_name][0]

    for sub_name in subs:

        pac_list = []

        analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
        pac_save_path = os.path.join(analysis_dir, 'source', 'PAC')
        pac_data = np.load(os.path.join(pac_save_path, f"PAC_MI_SOURCE_{sub_name[-5:]}{'_' + task}{'_' + task_stage}{block_name}.npy"))
        pac_data_t = np.transpose(pac_data, (1, 0, 2))

        ### NAN Imputation for PAC Matrices ###
        pac_imputed = pac_data_t.copy()

        # Step 1: Detect vertices with any NaN in their 20×20 PAC
        nan_mask = np.isnan(pac_data_t).any(axis=(1, 2))  # shape (5124,)

        if nan_mask.any() == True:
            # print(f"Found {nan_mask.sum()} vertices with NaN values.")

            # Step 2: Impute NaNs from neighbors
            for vtx in np.where(nan_mask)[0]:
                neighbors = adj[[vtx]].indices
                valid_neighbors = [n for n in neighbors if not nan_mask[n]]
                # Average PAC matrices from neighbors
                pac_imputed[vtx] = np.nanmean(pac_data_t[valid_neighbors], axis=0)

        # pac_zscore = zscore(pac_imputed, axis=0, nan_policy='omit')
        pac_list.append(pac_imputed)

    # Stack them along a new first axis (subject axis)
    pac_all = np.stack(pac_list, axis=0)
    # print('PAC array shape:', pac_all.shape)
    pac_map = pac_all.mean(axis=(0, 2, 3))
    # print('Averaged PAC map shape:', pac_map.shape)
    stc_pac._data = pac_map[:, None]   # (n_vertices_total, 1)
    # print('STC data shape:', stc_pac.data.shape)
    
    # Copy the STC to avoid modifying the original
    stc_sig = copy.deepcopy(stc_pac)
    stc_sig.data[~sig_mask] = 0.0

    clim = dict(
        kind="value",
        lims=[
            np.nanmin(pac_map), 
            (np.nanmax(pac_map) - np.nanmin(pac_map)) / 2 + np.nanmin(pac_map), 
            np.nanmax(pac_map)
            ],   # [min, mid, max]
        )
    # view = [["dorsal", "lateral"] if sig_cond == 'Y_BL_plan_' else ["dorsal", "parietal"]]
    brain = stc_sig.plot(subject="fsaverage_bem",
                initial_time=0.0,
                clim=clim,
                cortex="low_contrast",
                colormap='hot', # 'plasma' for raw MI, 'cool' for zscored MI
                hemi='both',
                views=[["dorsal", "lateral"] if sig_cond == 'Y_BL_plan_' else ["dorsal", "parietal"]][0],
                background='white',
                title = title,
                )
    brain.save_image(os.path.join(figs_save_path, title + '.png'))

______
Plotting drafts

In [ ]:
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
group = 'Y'
figs_save_path = os.path.join(eeg_data_dir, f'{group} group', 'sources', 'figs')
check_paths(figs_save_path)


sub_name = 's1_pac_sub07'
analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
task = '_MAIN'
task_stage = '_plan'
block_name = '_baseline'

stcs_path = os.path.join(analysis_dir, 'source', 'morphed_stcs', task, task_stage, block_name)

stcs = []
stcs_data = []

tmin = 0.0
tmax = 0.495

for stc_file in os.listdir(stcs_path):
    if stc_file.endswith('-rh.stc'): # MNE will load both hemispheres anyway
        stc_path = os.path.join(stcs_path, stc_file)
        stc = mne.read_source_estimate(stc_path, subject=sub_name)
        stc.crop(tmin=tmin, tmax=tmax)
        stcs.append(stc)
        stcs_data.append(stc.data)

source_array = np.stack(stcs_data, axis=0)
source_array.shape # (epochs x vertices x time)

(47, 5124, 249)

In [ ]:
# mri_data_dir = f'D:\\BonoKat\\research project\\# study 1\\mri_data\\fs_output\\freesurfer\\sub_dir\\Y'
# fname_fsaverage_src = os.path.join(mri_data_dir, "fsaverage_bem", "bem", "fsaverage-ico4-src.fif")

# eeg_data_dir = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set', group)
# src_to = mne.read_source_spaces(fname_fsaverage_src)

# # Morph the source estimates to the fsaverage space
# morph = mne.compute_source_morph(
#     stc,
#     subject_from=sub_name,
#     subject_to="fsaverage_bem",
#     src_to=src_to,
#     subjects_dir=mri_data_dir,
#     smooth=5 # small smoothing to avoid artifacts in the morphed data
# )

# stc_fsaverage = morph.apply(stc)
# stc.plot()
# stc_fsaverage.plot()
# stc_fsaverage

<SourceEstimate | 5124 vertices, subject : fsaverage_bem, tmin : 0.0 (ms), tmax : 496.0 (ms), tstep : 2.0 (ms), data shape : (5124, 249), ~4.9 MiB>

In [ ]:
analysis_dir = os.path.join(eeg_data_dir, group, sub_name, 'preproc', 'analysis')
pac_save_path = os.path.join(analysis_dir, 'source', 'PAC')
pac_data = np.load(os.path.join(pac_save_path, f"PAC_MI_SOURCE_{sub_name[-5:]}{task}{task_stage}{block_name}.npy"))

pac_data_t = np.transpose(pac_data, (1, 0, 2))

### NAN Imputation for PAC Matrices ###
pac_imputed = pac_data_t.copy()

# Step 1: Detect vertices with any NaN in their 20×20 PAC
nan_mask = np.isnan(pac_data_t).any(axis=(1, 2))  # shape (5124,)

if nan_mask.any() == True:
    print(f"Found {nan_mask.sum()} vertices with NaN values.")

    # Step 2: Impute NaNs from neighbors
    for vtx in np.where(nan_mask)[0]:
        neighbors = adj[[vtx]].indices
        valid_neighbors = [n for n in neighbors if not nan_mask[n]]
        # Average PAC matrices from neighbors
        pac_imputed[vtx] = np.nanmean(pac_data_t[valid_neighbors], axis=0)

pac_zscore = zscore(pac_imputed, axis=0, nan_policy='omit')
pac_map = pac_zscore.mean(axis=(1, 2))
pac_map_clean = np.nan_to_num(pac_map, nan=1, posinf=0.0, neginf=0.0)
stc_ref = stcs[0]  # any one of your loaded STCs
stc_ref_mean = stc_ref.copy().mean()
stc_pac = stc_ref_mean.copy()
stc_pac._data = pac_map_clean[:, None]   # (n_vertices_total, 1)
stc_pac.subject = "fsaverage_bem"
stc_pac

Found 3 vertices with NaN values.


<SourceEstimate | 5124 vertices, subject : fsaverage_bem, tmin : 249.0 (ms), tmax : 249.0 (ms), tstep : 498.0 (ms), data shape : (5124, 1), ~80 KiB>

In [ ]:
clim = dict(
    kind="value",
    lims=[np.nanmin(pac_map), (np.nanmax(pac_map) - np.nanmin(pac_map)) / 2 + np.nanmin(pac_map), np.nanmax(pac_map)],   # [min, mid, max]
    
)

views = {'lateral': ["lateral", "medial"],
         'dor-vent': ["dorsal", "ventral"]}

for key, value in views.items():
    title = f"{sub_name[-5:]}{task}{task_stage}{block_name}_z-score_PAC_{key}"

    brain = stc_pac.plot(subject="fsaverage_bem",
                initial_time=0.0,
                clim=clim,
                cortex="low_contrast",
                colormap='plasma', # 'plasma' for raw MI, 'cool' for zscored MI
                hemi='both',
                views=value, # "dorsal", "ventral"
                background='white',
                title = title
                )

    # brain.save_image(os.path.join(figs_save_path, title))

In [ ]:
## Plotting with ROI labels, if needed

# Select a few ROIs (names depend on your parcellation)
labels = mne.read_labels_from_annot('fsaverage_bem', parc='aparc')
roi_names = ["postcentral-lh", "precentral-lh", "superiorfrontal-lh"]
rois = [lab for lab in labels if lab.name in roi_names]

brain = stc_pac.plot(
    subject="fsaverage_bem",
    hemi="both",
    views=["lateral", "medial"],
    cortex="bone",
    clim=clim,
    colormap="plasma",
    initial_time=0.0,
)

# Add them in different colors
colors = ["yellow", "cyan", "magenta"]
for roi, color in zip(rois, colors):
    brain.add_label(roi, borders=True, color=color)

Reading labels from parcellation...
   read 35 labels from D:\BonoKat\research project\# study 1\mri_data\Y\fs\fsaverage_bem\label\lh.aparc.annot
   read 34 labels from D:\BonoKat\research project\# study 1\mri_data\Y\fs\fsaverage_bem\label\rh.aparc.annot


_____